<a href="https://colab.research.google.com/github/ffreemt/app1/blob/master/wip_ycco_2bit_AQLM_Mixtral_8x7B_Huang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 https://freedium.cfd/https://levelup.gitconnected.com/the-2-bit-quantization-is-insane-see-how-to-run-mixtral-8x7b-on-free-tier-colab-2803e39b9b9d  **The 2-bit Quantization is Insane! See How to Run Mixtral-8x7B on Free-tier Colab**

In [ ]:
!pip install aqlm[gpu]==1.0.1 -q
!pip install git+https://github.com/huggingface/accelerate.git@main -q
!pip install git+https://github.com/BlackSamorez/transformers.git@aqlm -q

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
quantized_model = AutoModelForCausalLM.from_pretrained(
    "BlackSamorez/Mixtral-8x7b-AQLM-2Bit-1x16-hf-test-dispatch",
    torch_dtype="auto", device_map="auto", low_cpu_mem_usage=True,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("Mixtral-8x7B-v0.1")

In [ ]:
%%time
output = quantized_model.generate(tokenizer("The relationship between humans and AI  ", return_tensors="pt")["input_ids"].cuda(), min_new_tokens=128, max_new_tokens=128)
print(tokenizer.decode(output[0]))

In [ ]:
import json
import textwrap

system_prompt = "A chat between a curious user and a blog writing assistant. "

def get_prompt(human_prompt):
    prompt_template=f"{system_prompt}\n\nUSER: {human_prompt} \nASSISTANT: "
    return prompt_template

In [ ]:
def remove_human_text(text):
    return text.split('USER:', 1)[0]

def parse_text(data):
    for item in data:
        text = item['generated_text']
        assistant_text_index = text.find('ASSISTANT:')
        if assistant_text_index != -1:
            assistant_text = text[assistant_text_index+len('ASSISTANT:'):].strip()
            assistant_text = remove_human_text(assistant_text)
            wrapped_text = textwrap.fill(assistant_text, width=100)
            print("#####", wrapped_text)
            # return assistant_text

In [ ]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=quantized_model,
    tokenizer=tokenizer,
    max_length=1200,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
)

In [ ]:
%%time
prompt = 'Write a short and engaging blog post of traveling in Bohol Island. '
raw_output = pipe(get_prompt(prompt))

parse_text(raw_output)

In [ ]:
!nvidia-smi

In [ ]:
%%time
prompt = '''Write a short python code to calculate factorial.
          '''
raw_output = pipe(get_prompt(prompt))

parse_text(raw_output)